In [1]:
from tqdm import tqdm
import pickle
import numpy as np
import cv2
import os
import os.path as osp
import shutil

In [2]:
label2idx = {
    'boeing747': 0,
    'boeing737': 0,
    'a321': 0,
    'airplane': 0,
    'a350': 0,
    'boeing777': 0,
    'c919': 0,
    'a330': 0,
    'other-airplane': 0,
    'arj21': 0,
    'a220': 0,
    'plane': 0,
    'boeing787': 0,
    'engineering-ship': 1,
    'other-ship': 1,
    'ship': 1,
    'passenger-ship': 2,
    'dry-cargo-ship': 3,
    'liquid-cargo-ship': 3,
    'warship': 4,
    'tugboat': 5,
    'fishing-boat': 5,
    'other-vehicle': 6,
    'large-vehicle': 6,
    'vehicle': 6,
    'small-vehicle': 7,
    'small-car': 7,
    'bus': 8,
    'tractor': 9,
    'truck-tractor': 9,
    'van': 10,
    'trailer': 11,
    'dump-truck': 12,
    'cargo-truck': 13,
    'golffield': 14,
    'baseball-field': 15,
    'baseballfield': 15,
    'baseball-diamond': 15,
    'soccer-ball-field': 16,
    'football-field': 17,
    'groundtrackfield': 18,
    'ground-track-field': 18,
    'basketballcourt': 19,
    'basketball-court': 19,
    'tennis-court': 20,
    'tenniscourt': 20,
    'storage-tank': 21,
    'storagetank': 21,
    'stadium': 22,
    'dam': 23,
    'intersection': 24,
    'swimming-pool': 25,
    'helipad': 26,
    'harbor': 27,
    'expressway-toll-station': 28,
    'bridge': 29,
    'chimney': 30,
    'roundabout': 31,
    'container-crane': 32,
    'overpass': 33,
    'motorboat': 34,
    'helicopter': 35,
    'expressway-service-area': 36,
    'trainstation': 37,
    'excavator': 38,
    'windmill': 39,
    'airport': 40,
}

idx2label = ['airplane', 'ship', 'passenger ship', 'cargo ship', 'warship', 'boat', 'car', 'small car', 'bus', 'tractor', 'van', 'trailer', 'dump truck', 'cargo truck', 'golf field', 'baseball field', 'soccer field', 'football field', 'ground track field', 'basketball court', 'tennis court', 'storage tank', 'stadium', 'dam', 'intersection', 'swimming pool', 'helipad', 'harbor', 'expressway toll station', 'bridge', 'chimney', 'roundabout', 'container crane', 'overpass', 'motorboat', 'helicopter', 'expressway service area', 'train station', 'excavator', 'windmill', 'airport']
idx2label = {i: l for i, l in enumerate(idx2label)}

print(label2idx['helicopter'], idx2label[35])

35 helicopter


In [29]:
def extract_bbox(root_path, out_path):
    corrupted_files = []
    os.makedirs(out_path, exist_ok=True)

    for path in tqdm(os.listdir(osp.join(root_path, 'ins'))):
        if not path.endswith('.pkl'):
            continue

        label = pickle.load(open(osp.join(root_path, 'ins', path), 'rb'))
        with open(osp.join(out_path, path.replace('.pkl', '.txt')), 'w') as f:
            try:
                for object in label:
                    if 'bbox' in object:
                        bbox = object['bbox'].astype(int)
                    elif 'rhbox' in object:
                        bbox = object['rhbox'].astype(int)
                    
                    h, w = object['mask']['size']
                    bbox[0] = max(0, bbox[0])
                    bbox[1] = max(0, bbox[1])
                    bbox[2] = min(w, bbox[2])
                    bbox[3] = min(h, bbox[3])
                    
                    category = object['category'].lower()
                    class_idx = label2idx[category]

                    f.write(f'{class_idx} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n')
            except Exception as e:
                corrupted_files.append(path)
    return corrupted_files

In [14]:
corrupted_files = extract_bbox('/home/hjh/dataset/SAMRS/SIOR', '/home/hjh/dataset/SAMRS/SIOR/bbox')
print(corrupted_files)
for file in corrupted_files:
    if  osp.exists(osp.join('/home/hjh/dataset/SAMRS/SIOR/bbox', file.replace('.pkl', '.txt'))):
        os.remove(osp.join('/home/hjh/dataset/SAMRS/SIOR/bbox', file.replace('.pkl', '.txt')))

100%|██████████████████████████████████████████████████| 23463/23463 [00:01<00:00, 13512.25it/s]

[]


In [27]:
corrupted_files = extract_bbox('/home/hjh/dataset/SAMRS/FAST', '/home/hjh/dataset/SAMRS/FAST/bbox')
print(corrupted_files)
for file in corrupted_files:
    if osp.exists(osp.join('/home/hjh/dataset/SAMRS/FAST/bbox', file.replace('.pkl', '.txt'))):
        os.remove(osp.join('/home/hjh/dataset/SAMRS/FAST/bbox', file.replace('.pkl', '.txt')))

100%|██████████████████████████████████████████████████| 64147/64147 [00:06<00:00, 10460.73it/s]

[]


In [ ]:
corrupted_files = extract_bbox('/home/hjh/dataset/SAMRS/SOTA', '/home/hjh/dataset/SAMRS/SOTA/bbox')
print(corrupted_files)
for file in corrupted_files:
    if osp.exists(osp.join('/home/hjh/dataset/SAMRS/SOTA/label', file.replace('.pkl', '.txt'))):
        os.remove(osp.join('/home/hjh/dataset/SAMRS/SOTA/label', file.replace('.pkl', '.txt')))

In [8]:
all_samples = np.array(os.listdir('/home/hjh/dataset/SAMRS/paint/images'))
np.random.shuffle(all_samples)

In [9]:
test_samples = all_samples[-5000:]
remaining_samples = all_samples[:-5000]
val_samples = remaining_samples[-5000:]
train_samples = remaining_samples[:-5000]
print('Train samples:', len(train_samples))
print('Val samples:', len(val_samples))
print('Test samples:', len(test_samples))

Train samples: 95090
Val samples: 5000
Test samples: 5000


In [ ]:
for path in tqdm(train_samples):
    image_path = osp.join('/home/hjh/dataset/SAMRS/paint/images', path)
    label_path = osp.join('/home/hjh/dataset/SAMRS/paint/bbox', path.replace('.png', '.txt').replace('.jpg', '.txt'))
    if not osp.exists(image_path) or not osp.exists(label_path):
        continue

    dest_img_dir = '/home/hjh/dataset/SAMRS/paint/images/train'
    dest_label_dir = '/home/hjh/dataset/SAMRS/paint/bbox/train'
    os.makedirs(dest_img_dir, exist_ok=True)
    os.makedirs(dest_label_dir, exist_ok=True)

    dest_img = osp.join(dest_img_dir, path)
    dest_label = osp.join(dest_label_dir, path.replace('.png', '.txt').replace('.jpg', '.txt'))

    shutil.move(image_path, dest_img)
    shutil.move(label_path, dest_label)

100%|██████████████████████████████████████████████████████| 95090/95090 [00:03<00:00, 25796.74it/s]


In [ ]:
for path in tqdm(val_samples):
    image_path = osp.join('/home/hjh/dataset/SAMRS/paint/images', path)
    label_path = osp.join('/home/hjh/dataset/SAMRS/paint/bbox', path.replace('.png', '.txt').replace('.jpg', '.txt'))
    if not osp.exists(image_path) or not osp.exists(label_path):
        continue

    dest_img_dir = '/home/hjh/dataset/SAMRS/paint/images/validation'
    dest_label_dir = '/home/hjh/dataset/SAMRS/paint/bbox/validation'
    os.makedirs(dest_img_dir, exist_ok=True)
    os.makedirs(dest_label_dir, exist_ok=True)

    dest_img = osp.join(dest_img_dir, path)
    dest_label = osp.join(dest_label_dir, path.replace('.png', '.txt').replace('.jpg', '.txt'))

    shutil.move(image_path, dest_img)
    shutil.move(label_path, dest_label)

100%|████████████████████████████████████████████████████████| 5000/5000 [00:00<00:00, 25694.38it/s]


In [ ]:
for path in tqdm(test_samples):
    image_path = osp.join('/home/hjh/dataset/SAMRS/paint/images', path)
    label_path = osp.join('/home/hjh/dataset/SAMRS/paint/bbox', path.replace('.png', '.txt').replace('.jpg', '.txt'))
    if not osp.exists(image_path) or not osp.exists(label_path):
        continue

    dest_img_dir = '/home/hjh/dataset/SAMRS/paint/images/test'
    dest_label_dir = '/home/hjh/dataset/SAMRS/paint/bbox/test'
    os.makedirs(dest_img_dir, exist_ok=True)
    os.makedirs(dest_label_dir, exist_ok=True)

    dest_img = osp.join(dest_img_dir, path)
    dest_label = osp.join(dest_label_dir, path.replace('.png', '.txt').replace('.jpg', '.txt'))

    shutil.move(image_path, dest_img)
    shutil.move(label_path, dest_label)

100%|████████████████████████████████████████████████████████| 5000/5000 [00:00<00:00, 25942.27it/s]


In [8]:
corrupted = []
for label in tqdm(os.listdir('/home/hjh/dataset/SAMRS/paint/bbox/train')):
    with open(osp.join('/home/hjh/dataset/SAMRS/paint/bbox/train', label), 'r') as f:
        for line in f:
            items = line.strip().split(' ')
            class_idx = int(items[0])
            bbox = list(map(int, items[1:]))
            if bbox[0] >= bbox[2] or bbox[1] >= bbox[3]:
                print('Invalid bbox:', bbox, label) 
                corrupted.append(label)
                break

for label in tqdm(os.listdir('/home/hjh/dataset/SAMRS/paint/bbox/validation')):
    with open(osp.join('/home/hjh/dataset/SAMRS/paint/bbox/validation', label), 'r') as f:
        for line in f:
            items = line.strip().split(' ')
            class_idx = int(items[0])
            bbox = list(map(int, items[1:]))
            if bbox[0] >= bbox[2] or bbox[1] >= bbox[3]:
                print('Invalid bbox:', bbox, label) 
                corrupted.append(label)
                break

for label in tqdm(os.listdir('/home/hjh/dataset/SAMRS/paint/bbox/test')):
    with open(osp.join('/home/hjh/dataset/SAMRS/paint/bbox/test', label), 'r') as f:
        for line in f:
            items = line.strip().split(' ')
            class_idx = int(items[0])
            bbox = list(map(int, items[1:]))
            if bbox[0] >= bbox[2] or bbox[1] >= bbox[3]:
                print('Invalid bbox:', bbox, label) 
                corrupted.append(label)
                break

print(corrupted)

  8%|████▌                                                  | 7955/95090 [00:00<00:02, 39843.00it/s]

Invalid bbox: [0, 391, 0, 393] train_238_0001.txt


 34%|██████████████████▏                                   | 31928/95090 [00:00<00:01, 39639.56it/s]

Invalid bbox: [0, 557, 0, 559] train_83_0001.txt


 51%|███████████████████████████▍                          | 48394/95090 [00:01<00:01, 40057.92it/s]

...

Invalid bbox: [162, 329, 162, 330] 04137.txt
Invalid bbox: [69, 0, 81, 0] train_99_0000.txt


 88%|███████████████████████████████████████████████▋      | 83991/95090 [00:02<00:00, 38570.39it/s]

Invalid bbox: [138, 161, 138, 161] 16734.txt
Invalid bbox: [1, 786, 1, 788] 08325.txt


100%|████████████████████████████████████████████████████████| 5000/5000 [00:00<00:00, 39608.51it/s]

Invalid bbox: [530, 330, 530, 330] 15504.txt
Invalid bbox: [191, 535, 191, 535] 15906.txt
['train_238_0001.txt', 'train_83_0001.txt', '07007.txt', '04137.txt', 'train_99_0000.txt', '16734.txt', '08325.txt', '15504.txt', '15906.txt']
